In [1]:
import torch
import numpy as np
import time
import os
from utils import HDDataset
from model import BModel
from torchvision import datasets, transforms
import scipy
from encoder import RandomFourierEncoder
import main2_fmnist 
import importlib
importlib.reload(main2_fmnist)
import csv
from scipy.io import savemat, loadmat
from torch.utils.data import DataLoader, Subset, TensorDataset


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
gorder = 2
dim = 10000
gamma = 0.3
lr = 0.01
classes = 10
channels = 3
epochs = 3
train = main2_fmnist.train
test = main2_fmnist.test

In [ ]:
trainset = datasets.FashionMNIST(root='../../Data', train=True, download=True, transform=transforms.ToTensor())
testset = datasets.FashionMNIST(root='../../Data', train=False, download=True, transform=transforms.ToTensor())
n_splits = 20
split_size = len(testset) // n_splits
accuracies1 = np.zeros((11, n_splits))
hyperdims = loadmat('../EHDGNet_FashionMNIST_nHD.mat')['EHDGNet_FashionMNIST_nHD']
hyperdims = np.mean(hyperdims, axis=1, dtype=int)
print(hyperdims)
input_dim = torch.prod(torch.tensor(list(trainset[0][0].size())))
print(input_dim)
for i, nHDC in enumerate(hyperdims):
    print(f'nHDC: {nHDC} ------------')
    encoder = RandomFourierEncoder(input_dim, gamma, gorder, output_dim=nHDC)
    encoder.build_item_mem()
    train_hd, y_train = encoder.encode_data_extract_labels(trainset)
    train_dataset = HDDataset(train_hd, y_train)
    trainloader = DataLoader(train_dataset, batch_size=1024, shuffle=True, pin_memory=True, num_workers=2)
    for split_idx in range(n_splits):
        # Test
        print(f'Split Index: {split_idx+1}')
        start_idx = split_idx * split_size
        end_idx = start_idx + split_size
        split_subset = Subset(testset, range(start_idx, end_idx))
        test_hd, y_test = encoder.encode_data_extract_labels(split_subset)
        test_dataset = HDDataset(test_hd, y_test)
        testloader = DataLoader(test_dataset, batch_size=512, shuffle=False, pin_memory=True, num_workers=2)
        accuracies1[i, split_idx] = train(trainloader, testloader, lr, nHDC)
        del testloader
        del test_dataset
        del test_hd
        
    del train_hd
    del trainloader
    del train_dataset
    print(f'nHDC: {nHDC} ----------- Finished')
    print(f'Average Accuracy for nHDC={nHDC}: {np.mean(accuracies1[i, :])}')

[ 5000  7333  9000 11333 13333 15000 17333 19000 21333 23000 25000]
tensor(784)
nHDC: 5000 ------------
the threshold to discretize fourier features to group elements tensor([0.])
Split Index: 1
Split Index: 2
Split Index: 3
Split Index: 4
Split Index: 5
Split Index: 6
Split Index: 7
Split Index: 8
Split Index: 9
Split Index: 10
Split Index: 11
Split Index: 12
Split Index: 13


In [9]:
np.mean(accuracies1, axis=1)

array([73.97, 80.94, 81.12, 82.32, 82.08, 83.99, 83.02, 84.76, 84.73,
       85.3 , 84.53])

In [10]:
savemat('RFFHDC_FashionMNIST.mat', {'RFFHDC_FashionMNIST': accuracies1})